## IMPORT LIBRARY


In [3]:
# import libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cufflinks as cf
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler

## PREPARATION FOR MAIN DATA

In [ ]:
# INPUT STOCK DATA
ticker='BBCA.JK'
sampling=60
training_len=0.4

#start_date='1980-01-01'
end_date=datetime.today().strftime('%Y-%m-%d')
#end_date='2020-05-26'


df=web.DataReader(ticker,data_source='yahoo', end=end_date)
df['SMA1']=df['Close'].rolling(window=5).mean()
df=df[4:]
df

,High,Low,Open,Close,Volume,Adj Close,SMA1
Date,,,,,,,
2015-09-08,12000.0,11750.0,11825.0,11800.0,13236700.0,10931.146484,12110.0
2015-09-09,12025.0,11900.0,11900.0,11925.0,10665500.0,11046.943359,12040.0
2015-09-10,12025.0,11775.0,11850.0,11925.0,12626100.0,11046.943359,11965.0
2015-09-11,12000.0,11900.0,12000.0,11925.0,9006500.0,11046.943359,11900.0
2015-09-14,12125.0,11925.0,12000.0,12100.0,7167200.0,11209.054688,11935.0
...,...,...,...,...,...,...,...
2020-08-25,31900.0,31525.0,31600.0,31825.0,9842200.0,31825.000000,31775.0
2020-08-26,31950.0,31525.0,31925.0,31700.0,8596600.0,31700.000000,31710.0
2020-08-27,33000.0,31700.0,31825.0,33000.0,13665200.0,33000.000000,31950.0


In [ ]:
#df['Close'].iplot(asFigure=True, title=ticker + ' CLOSE PRICE',yTitle="HARGA PENUTUPAN (Rp)", xTitle="TANGGAL")

## PREDIKSI MOVING AVERAGE


In [ ]:
# convert the data inti numpy array
data=df.filter(['SMA1'])
dataset=data.values

# get the number of rows to train the model on
training_data_len=math.ceil(len(dataset)* training_len)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

# SPLIT DATA INTO x_train and y_train
train_data=scaled_data[0:training_data_len]
x_train=[]
y_train=[]

for i in range(sampling,len(train_data)):
  x_train.append(train_data[i-sampling:i,0])
  y_train.append(train_data[i,0])

x_train, y_train=np.array(x_train), np.array(y_train)

# SPLIT DATA INTO x_test and y_test
test_data=scaled_data[training_data_len-sampling:]

x_test=[]
y_test=dataset[training_data_len+1:]

for i in range(sampling, len(test_data)):
  x_test.append(test_data[i-sampling:i,0]) 

In [ ]:
# Linear regression MODEL FIT

lr=LinearRegression()
lr.fit(x_train,y_train)
  
print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

print('Training set score: {:.4f}'.format(lr.score(x_train,y_train)))
#print('Test set score: {:.4f}'.format(lr.score(x_test,y_test)))

lr.coef_: [-0.07996714  0.2138861  -0.16272143 -0.02347351  0.16417167 -0.30244716
  0.30314063 -0.0872143  -0.16832628  0.32369758 -0.33599163  0.22680733
 -0.09489509 -0.11300064  0.41240142 -0.3939098   0.11174576  0.00653543
 -0.23113719  0.57317262 -0.44506327  0.0856478   0.05602427 -0.37595254
  0.74908846 -0.52328418  0.14244805  0.02562705 -0.49070458  1.00477749
 -0.70280816  0.14324564  0.04253133 -0.55048372  1.06414879 -0.75103065
  0.2648633  -0.11315735 -0.44059607  1.10671328 -0.80713104  0.21306914
 -0.08108144 -0.47150713  1.25098032 -0.95777815  0.23017043  0.037555
 -0.64635026  1.34982853 -0.94632791  0.14925693  0.00991573 -0.62966018
  1.54612044 -0.99045027  0.09760078 -0.03753971 -0.76314137  1.81210607]
lr.intercept_: 0.00012051449337402431
Training set score: 0.9997


In [ ]:
predictions= lr.predict(x_test)
predictions=np.reshape(predictions,(predictions.shape[0],1))
predictions=scaler.inverse_transform(predictions)
#len(predictions)
#predictions

In [ ]:
df_baru=df[training_data_len:]
df_baru['prediction']=predictions
df_baru[['SMA1','prediction']].iplot(asFigure=True,title=ticker,legend='top').show()
df_baru.tail(5)

,High,Low,Open,Close,Volume,Adj Close,SMA1,prediction
Date,,,,,,,,
2020-08-25,31900.0,31525.0,31600.0,31825.0,9842200.0,31825.0,31775.0,31773.349673
2020-08-26,31950.0,31525.0,31925.0,31700.0,8596600.0,31700.0,31710.0,31656.169848
2020-08-27,33000.0,31700.0,31825.0,33000.0,13665200.0,33000.0,31950.0,31630.489285
2020-08-28,33000.0,32125.0,32800.0,32475.0,10458500.0,32475.0,32115.0,32143.479040
2020-08-31,32700.0,31375.0,32500.0,31375.0,19406400.0,31375.0,32075.0,32261.621209


In [ ]:
close=df_baru.filter(['SMA1']).values
predict=df_baru.filter(['prediction']).values
rmse =np.sqrt(np.mean(((close-predict)**2)))
print('PERSEN ERROR PREDIKSI SMA = {:.2f}%'.format(rmse))

PERSEN ERROR PREDIKSI SMA = 95.45%


In [ ]:
new_df=df.filter(['SMA1'])
last_60_days=new_df[-sampling:].values
last_data=scaler.transform(last_60_days)
last_data=np.reshape(last_data,(1,len(last_data)))
last_prediction=lr.predict(last_data)
last_prediction=[last_prediction]
last_prediction=scaler.inverse_transform(last_prediction)

print('PREDIKSI SMA ({}) BESOK = {}'.format(end_date,last_prediction[0][0]))

PREDIKSI SMA (2020-08-31) BESOK = 32129.34432488491


## CLOSE PRICE KESELURUHAN 
PAKE NILAI PREDIKSI

In [ ]:
df_close=df_baru[['Close','SMA1','prediction']]
df_close['y']=df_close['Close'].rolling(window=4).sum().shift()
df_close['% error sma pred']=(df_close['SMA1']-df_close['prediction'])/df_close['SMA1']*100
df_close=df_close[4:]
error=df_close['% error sma pred'].mean()
df_close['close prediction']=5*(df_close['prediction'])-df_close['y']
df_close[['Close','close prediction']].iplot(asFigure=True,title='PREDIKSI HARGA CLOSE '+ticker,legend='top').show()
df_close.tail()

,Close,SMA1,prediction,y,% error sma pred,close prediction
Date,,,,,,
2020-08-25,31825.0,31775.0,31773.349673,127050.0,0.005194,31816.748365
2020-08-26,31700.0,31710.0,31656.169848,126850.0,0.169758,31430.849241
2020-08-27,33000.0,31950.0,31630.489285,126750.0,1.000034,31402.446427
2020-08-28,32475.0,32115.0,32143.479040,128100.0,-0.088678,32617.395200
2020-08-31,31375.0,32075.0,32261.621209,129000.0,-0.581828,32308.106046


In [ ]:
harga_close=df_close.filter(['Close']).values
predict_close=df_close.filter(['close prediction']).values
rmse_close_price=np.sqrt(np.mean(((harga_close-predict_close)**2)))
print('PERSEN ERROR PREDIKSI HARGA CLOSE = {:.2f}%'.format(rmse_close_price))

PERSEN ERROR PREDIKSI HARGA CLOSE = 477.31%


## PREDIKSI CLOSE PRICE BESOK

In [ ]:
y_besok=df_close['Close'][-4:].sum()
close_price_besok=5*(last_prediction[0][0])-y_besok
close_price_besok

32096.721624424565